# Data Processing with DataFrame 

[DataFrames](https://docs.microsoft.com/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview) are an easy way to explore and manipulate data. We can load, convert, and combine data.  

Let's take a look at the following examples of hockey data. We have information on US and European players, plus a list of player salaries. Let's combine all of this into a single dataset for predicting salaries. Let's normalize the data of US and European players, combine them into one source, and then union with player salaries.  

## Load packages

Get data frame api, visualization and formatting.

In [1]:
// using nightly-build
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML, 2.0.0-preview.22356.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

## Inspect data

Let's get started loading our data on the players from the US into a DataFrame.

#### Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [1]:
using System.Linq;
using Microsoft.Data.Analysis;

var usaDataPath = EnsureDataSetDownloaded("usa_hockey.csv"); 
var usaDf = DataFrame.LoadCsv(usaDataPath);

usa_hockey.csv found here: C:\dev\csharp-notebooks\machine-learning\data\usa_hockey.csv


DataFrames store data as a collection of columns. This makes it easy to interact with the data. To get a preview of the columns, we'll run `Info()`.

Here we have a information about the players, where they were born, their height and weight, etc. We also have statistics on their play, like the number of goals/assists, points scored, and games played. 

We can already see that some fields have fewer values than other. Draft year is only available for 7 out of 10 players. 

In [1]:
usaDf.Info()

index,Info,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,DataType,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.String,System.String,System.String,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single
1,Length (excluding null values),10,10,10,10,7,7,10,10,10,10,10,10,10,10,10,10,10,10


Take a look at the actual data. Everything looks pretty normal, but there are some null values. 

In [1]:
usaDf

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,88-16-04,USA,72,218,2006,7,R,Okposo,Kyle,RW,BUF,65,19,26,45,24,1443,73983
1,90-08-10,USA,76,210,2009,114,L,Helgeson,Seth,D,N.J,9,1,0,1,15,177,7273
2,96-26-11,USA,77,203,2015,37,R,Carlo,Brandon,D,BOS,82,6,10,16,59,2080,102414
3,90-16-11,USA,74,219,<null>,<null>,L,Schaller,Tim,C,BOS,59,7,7,14,23,1035,43436
4,92-20-03,USA,72,215,2010,37,R,Faulk,Justin,D,CAR,75,17,20,37,32,1987,104133
5,94-01-05,USA,74,205,2012,120,L,Slavin,Jaccob,D,CAR,82,5,29,34,12,2135,115316
6,90-20-06,USA,75,221,2008,128,R,Pateryn,Greg,D,DAL/MTL,36,1,8,9,10,720,33312
7,90-27-05,USA,74,196,2009,198,R,Dowd,Nic,C,L.A,70,6,16,22,25,1230,52314
8,90-16-07,USA,75,221,<null>,<null>,L,Lashoff,Brian,D,DET,5,0,0,0,0,93,3754
9,86-09-08,USA,71,197,<null>,<null>,R,Cannone,Patrick,C,MIN,3,0,0,0,0,35,1419


### Europe data

We also have data for some European players. Let's preview the data and see how it's different. 

In [1]:
var europeDataPath = EnsureDataSetDownloaded("europe_hockey.csv"); 
var europeDf = DataFrame.LoadCsv(europeDataPath);
europeDf.Info()

index,Info,Birthday,Nat,Height_cm,Weight_kg,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,DataType,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.String,System.String,System.String,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single
1,Length (excluding null values),10,10,10,10,8,8,10,10,10,10,10,10,10,10,10,10,10,10


The values look very similar to the US players. One difference is height is in centimeters and weight is in kilograms. Some of the values in this file are missing too. 

In [1]:
europeDf

index,Birthday,Nat,Height_cm,Weight_kg,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,90-11-21,SWE,180,84,2009,53,L,Rodin,Anton,RW,VAN,3,0,1,1,0,40,1604
1,86-04-28,CZE,188,107,2004,180,R,Polak,Roman,D,TOR,75,4,7,11,65,1817,80643
2,90-11-29,CZE,201,100,<null>,<null>,R,Sustr,Andrej,D,T.B,80,3,11,14,43,1813,84427
3,96-05-25,CZE,183,82,2014,25,R,Pastrnak,David,RW/LW,BOS,75,34,36,70,34,1597,80921
4,94/03/23,SWE,190,95,2012,81,R,Sundqvist,Oskar,C,PIT,10,0,0,0,2,138,5504
5,90-06-05,CZE,183,93,2010,66,R,Gudas,Radko,D,PHI,67,6,17,23,93,1739,77555
6,90-06-01,CHE,185,91,2008,38,L,Josi,Roman,D,NSH,72,12,37,49,18,2076,108323
7,86-08-01,SWE,180,86,2005,216,R,Stralman,Anton,D,T.B,73,5,17,22,20,1963,100304
8,92-11-12,SWE,190,98,2011,4,R,Larsson,Adam,D,EDM,79,4,15,19,55,2055,95509
9,89-11-21,FRA,183,91,<null>,<null>,L,Roussel,Antoine,LW,DAL,60,12,15,27,115,1427,55866


Looking at the acutal data we can see that birthday is in a different format YY-MM-DD format. 

Let's dive in a little further and see which countries are represented here. 

In [1]:
europeDf["Nat"].ValueCounts()

index,Values,Counts
0,SWE,4
1,CZE,4
2,CHE,1
3,FRA,1


## Convert data

Now that we know something about the data let's normalize and combine these data sources into one. 

Europe players have height and weight in kilograms and centimeters. Since these are players are in the NHL, let's convert these values into pounds and inches to match the American players.

In [1]:
europeDf["Weight_kg"] = europeDf["Weight_kg"].Multiply(2.20462262).Round(); 
europeDf["Height_cm"] = europeDf["Height_cm"].Divide(2.54).Round();

europeDf["Weight_kg"].SetName("Weight"); 
europeDf["Height_cm"].SetName("Height");

Let's also convert the European birthday to the American format for consistency. 

In [1]:
// There are birthdays in the format "1999/1/1" and "1999-1-1". Include both splitter characters. 
char[] delimiterChars = { '-', '/'};

var birthday = 
    ((IEnumerable<string>)europeDf["Birthday"])
        .Select(x => x.Split(delimiterChars));

europeDf["Birthday"] = new StringDataFrameColumn("Birthday", birthday.Select(x => x[0] + "-" + x[2] + "-" + x[1]));

To combine the DataFrames they need to look identical. Let's sample some data and confirm the shape is the same. 

In [1]:
europeDf.Sample(1)

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,96-25-05,CZE,72,181,2014,25,R,Pastrnak,David,RW/LW,BOS,75,34,36,70,34,1597,80921


In [1]:
usaDf.Sample(1)

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,90-27-05,USA,74,196,2009,198,R,Dowd,Nic,C,L.A,70,6,16,22,25,1230,52314


## Combine data sources

In [1]:
var allPlayersDf = usaDf.Clone();
europeDf.Rows.ToList<DataFrameRow>().ForEach(row => {allPlayersDf.Append(row, /*append in place*/ true);})

We now have one DataFrame with all the player information. 

In [1]:
allPlayersDf

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,88-16-04,USA,72,218,2006,7,R,Okposo,Kyle,RW,BUF,65,19,26,45,24,1443,73983
1,90-08-10,USA,76,210,2009,114,L,Helgeson,Seth,D,N.J,9,1,0,1,15,177,7273
2,96-26-11,USA,77,203,2015,37,R,Carlo,Brandon,D,BOS,82,6,10,16,59,2080,102414
3,90-16-11,USA,74,219,<null>,<null>,L,Schaller,Tim,C,BOS,59,7,7,14,23,1035,43436
4,92-20-03,USA,72,215,2010,37,R,Faulk,Justin,D,CAR,75,17,20,37,32,1987,104133
5,94-01-05,USA,74,205,2012,120,L,Slavin,Jaccob,D,CAR,82,5,29,34,12,2135,115316
6,90-20-06,USA,75,221,2008,128,R,Pateryn,Greg,D,DAL/MTL,36,1,8,9,10,720,33312
7,90-27-05,USA,74,196,2009,198,R,Dowd,Nic,C,L.A,70,6,16,22,25,1230,52314
8,90-16-07,USA,75,221,<null>,<null>,L,Lashoff,Brian,D,DET,5,0,0,0,0,93,3754
9,86-09-08,USA,71,197,<null>,<null>,R,Cannone,Patrick,C,MIN,3,0,0,0,0,35,1419


The lists are combined together, but we have some missing data for Draft Year and Overall Draft position. Let's replace those nulls so they don't mess up our training. 

In [1]:
// We're missing information in DraftYear and OverallDraft
allPlayersDf["DraftYear"] = allPlayersDf["DraftYear"].FillNulls(2000); 

var medianDraft = allPlayersDf["OverallDraft"].Median();
allPlayersDf["OverallDraft"] = allPlayersDf["OverallDraft"].FillNulls(medianDraft); 

allPlayersDf

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Last Name,First Name,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce
0,88-16-04,USA,72,218,2006,7,R,Okposo,Kyle,RW,BUF,65,19,26,45,24,1443,73983
1,90-08-10,USA,76,210,2009,114,L,Helgeson,Seth,D,N.J,9,1,0,1,15,177,7273
2,96-26-11,USA,77,203,2015,37,R,Carlo,Brandon,D,BOS,82,6,10,16,59,2080,102414
3,90-16-11,USA,74,219,2000,59.5,L,Schaller,Tim,C,BOS,59,7,7,14,23,1035,43436
4,92-20-03,USA,72,215,2010,37,R,Faulk,Justin,D,CAR,75,17,20,37,32,1987,104133
5,94-01-05,USA,74,205,2012,120,L,Slavin,Jaccob,D,CAR,82,5,29,34,12,2135,115316
6,90-20-06,USA,75,221,2008,128,R,Pateryn,Greg,D,DAL/MTL,36,1,8,9,10,720,33312
7,90-27-05,USA,74,196,2009,198,R,Dowd,Nic,C,L.A,70,6,16,22,25,1230,52314
8,90-16-07,USA,75,221,2000,59.5,L,Lashoff,Brian,D,DET,5,0,0,0,0,93,3754
9,86-09-08,USA,71,197,2000,59.5,R,Cannone,Patrick,C,MIN,3,0,0,0,0,35,1419


Now that our player information looks good, let's take a look at the salary data we're trying to union with.

This file is in JSON format. DataFrames don't support loading from JSON yet. There is a work around. It's fairly easy to convert from IDataView to DataFrame. Let's use that method to get what we need. 

In [1]:
public class PlayerSalary
{
    public string Name { get; set; }

    public float Salary { get; set; }
}

In [1]:
using System.Text.Json;
using System.IO; 
using Microsoft.ML; 

// Read in JSON file
string jsonString = File.ReadAllText(@"data/playerSalary.json");
var players = JsonSerializer.Deserialize<List<PlayerSalary>>(jsonString);

// Load it into an IDataView
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromEnumerable<PlayerSalary>(players);

// Convert to a DataFrame
var playerSalaryDf = data.ToDataFrame();

playerSalaryDf

index,Name,Salary
0,Adam Larsson,3000000
1,Andrej Sustr,1600000
2,Antoine Roussel,2200000
3,Anton Rodin,950000
4,Anton Stralman,4500000
5,Brandon Carlo,717500
6,Brian Lashoff,650000
7,David Pastrnak,925000
8,Greg Pateryn,750000
9,Jaccob Slavin,742500


### Convert two columns into one

Our player salary information has the name format "First Last". The player details has two separate columns for first and last name. Let's fix this. 

In [1]:
// Grab first and last name. Zip them together into one list. 
var firstNames = (IEnumerable<string>)allPlayersDf["First Name"]; 
var lastNames = (IEnumerable<string>)allPlayersDf["Last Name"];
var fullNames = firstNames.Zip(lastNames, (first, last) => first + " " + last); 

// Create the new column from the combined names
allPlayersDf["FullName"] = new StringDataFrameColumn("FullName", fullNames); 

// Cleanup the unneeded first and last name columns 
allPlayersDf.Columns.Remove("First Name"); 
allPlayersDf.Columns.Remove("Last Name"); 

### Merge two DataFrames together
 
Join the player salaries with player information. We will join on the Name information. 

In [1]:
// Merge the player salary information with the player details. 
var allPlayersWithSalaries = allPlayersDf.Merge(playerSalaryDf, new string[] {"FullName"}, new string[] {"Name"});

// We now have two columns with names. Remove one. 
allPlayersWithSalaries.Columns.Remove("Name");

// Just for fun, let's put them in alphabetical order 
allPlayersWithSalaries = allPlayersWithSalaries.OrderBy("FullName"); 

In [1]:
allPlayersWithSalaries

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce,FullName,Salary
0,92-12-11,SWE,75,216,2011,4,R,D,EDM,79,4,15,19,55,2055,95509,Adam Larsson,3000000
1,90-29-11,CZE,79,220,2000,59.5,R,D,T.B,80,3,11,14,43,1813,84427,Andrej Sustr,1600000
2,89-21-11,FRA,72,201,2000,59.5,L,LW,DAL,60,12,15,27,115,1427,55866,Antoine Roussel,2200000
3,90-21-11,SWE,71,185,2009,53,L,RW,VAN,3,0,1,1,0,40,1604,Anton Rodin,950000
4,86-01-08,SWE,71,190,2005,216,R,D,T.B,73,5,17,22,20,1963,100304,Anton Stralman,4500000
5,96-26-11,USA,77,203,2015,37,R,D,BOS,82,6,10,16,59,2080,102414,Brandon Carlo,717500
6,90-16-07,USA,75,221,2000,59.5,L,D,DET,5,0,0,0,0,93,3754,Brian Lashoff,650000
7,96-25-05,CZE,72,181,2014,25,R,RW/LW,BOS,75,34,36,70,34,1597,80921,David Pastrnak,925000
8,90-20-06,USA,75,221,2008,128,R,D,DAL/MTL,36,1,8,9,10,720,33312,Greg Pateryn,750000
9,94-01-05,USA,74,205,2012,120,L,D,CAR,82,5,29,34,12,2135,115316,Jaccob Slavin,742500


Everything looks really good, expect we have on player without a salary. That won't be useful to us. Let's drop this null value from the table. 

In [1]:
allPlayersWithSalaries = allPlayersWithSalaries.DropNulls();
allPlayersWithSalaries

index,Birthday,Nat,Height,Weight,DraftYear,OverallDraft,Hand,Position,Team,GamesPlayed,Goals,Assists,Points,PIM,Shifts,TimeOnIce,FullName,Salary
0,92-12-11,SWE,75,216,2011,4,R,D,EDM,79,4,15,19,55,2055,95509,Adam Larsson,3000000
1,90-29-11,CZE,79,220,2000,59.5,R,D,T.B,80,3,11,14,43,1813,84427,Andrej Sustr,1600000
2,89-21-11,FRA,72,201,2000,59.5,L,LW,DAL,60,12,15,27,115,1427,55866,Antoine Roussel,2200000
3,90-21-11,SWE,71,185,2009,53,L,RW,VAN,3,0,1,1,0,40,1604,Anton Rodin,950000
4,86-01-08,SWE,71,190,2005,216,R,D,T.B,73,5,17,22,20,1963,100304,Anton Stralman,4500000
5,96-26-11,USA,77,203,2015,37,R,D,BOS,82,6,10,16,59,2080,102414,Brandon Carlo,717500
6,90-16-07,USA,75,221,2000,59.5,L,D,DET,5,0,0,0,0,93,3754,Brian Lashoff,650000
7,96-25-05,CZE,72,181,2014,25,R,RW/LW,BOS,75,34,36,70,34,1597,80921,David Pastrnak,925000
8,90-20-06,USA,75,221,2008,128,R,D,DAL/MTL,36,1,8,9,10,720,33312,Greg Pateryn,750000
9,94-01-05,USA,74,205,2012,120,L,D,CAR,82,5,29,34,12,2135,115316,Jaccob Slavin,742500


## Save results

We succesfully combined three different sources into one. Now it's time to save our results into a .csv file that can be used for training. 

In [1]:
DataFrame.WriteCsv(allPlayersWithSalaries, "allPlayers.csv", ',')